- modifica features
- aggiunta di ensamble models

In [1]:
import json
import pandas as pd
import os
import numpy as np

# --- Define the path to our data ---
COMPETITION_NAME = 'fds-pokemon-battles-prediction-2025'
DATA_PATH = os.path.join('../input', COMPETITION_NAME)

train_file_path = os.path.join(DATA_PATH, 'train.jsonl')
test_file_path = os.path.join(DATA_PATH, 'test.jsonl')

In [13]:
train_data = []

# Read the file line by line
print(f"Loading data from '{train_file_path}'...")
try:
    with open(train_file_path, 'r') as f:
        for line in f:
            # json.loads() parses one line (one JSON object) into a Python dictionary
            train_data.append(json.loads(line))
        print(f"Successfully loaded {len(train_data)} battles.")
    # Let's inspect the first battle to see its structure
    print("\n--- Structure of the first train battle: ---")
    if train_data:
        first_battle = train_data[7]
        
        # To keep the output clean, we can create a copy and truncate the timeline
        battle_for_display = first_battle.copy()
        battle_for_display['battle_timeline'] = battle_for_display.get('battle_timeline', [])#[:2] # Show first 2 turns
        
        # Use json.dumps for pretty-printing the dictionary
        print(json.dumps(battle_for_display, indent=4))
        if len(first_battle.get('battle_timeline', [])) > 3:
            print("    ...")
            print("    (battle_timeline has been truncated for display)")


except FileNotFoundError:
    print(f"ERROR: Could not find the training file at '{train_file_path}'.")
    print("Please make sure you have added the competition data to this notebook.")

Loading data from '../input/fds-pokemon-battles-prediction-2025/train.jsonl'...
Successfully loaded 10000 battles.

--- Structure of the first train battle: ---
{
    "player_won": true,
    "p1_team_details": [
        {
            "name": "alakazam",
            "level": 100,
            "types": [
                "notype",
                "psychic"
            ],
            "base_hp": 55,
            "base_atk": 50,
            "base_def": 45,
            "base_spa": 135,
            "base_spd": 135,
            "base_spe": 120
        },
        {
            "name": "exeggutor",
            "level": 100,
            "types": [
                "grass",
                "psychic"
            ],
            "base_hp": 95,
            "base_atk": 95,
            "base_def": 85,
            "base_spa": 125,
            "base_spd": 125,
            "base_spe": 55
        },
        {
            "name": "starmie",
            "level": 100,
            "types": [
                "psychi

In [3]:
test_data = []

print(f"Loading data from '{test_file_path}'...")
try:
    with open(test_file_path, 'r') as f:
        for line in f:
            test_data.append(json.loads(line))
    
    print("\n--- Structure of the first test battle: ---")
    if test_data:
            first_test_battle = test_data[0]
            
            # To keep the output clean, we can create a copy and truncate the timeline
            test_battle_for_display = first_test_battle.copy()
            test_battle_for_display['battle_timeline'] = test_battle_for_display.get('battle_timeline', [])[:5] # Show first 2 turns
            
            # Use json.dumps for pretty-printing the dictionary
            print(json.dumps(test_battle_for_display, indent=4))
            if len(first_test_battle.get('battle_timeline', [])) > 3:
                print("    ...")
                print("    (battle_timeline has been truncated for display)")


except FileNotFoundError:
    print(f"ERROR: Could not find the training file at '{test_file_path}'.")
    print("Please make sure you have added the competition data to this notebook.")

Loading data from '../input/fds-pokemon-battles-prediction-2025/test.jsonl'...

--- Structure of the first test battle: ---
{
    "p1_team_details": [
        {
            "name": "gengar",
            "level": 100,
            "types": [
                "ghost",
                "poison"
            ],
            "base_hp": 60,
            "base_atk": 65,
            "base_def": 60,
            "base_spa": 130,
            "base_spd": 130,
            "base_spe": 110
        },
        {
            "name": "chansey",
            "level": 100,
            "types": [
                "normal",
                "notype"
            ],
            "base_hp": 250,
            "base_atk": 5,
            "base_def": 5,
            "base_spa": 105,
            "base_spd": 105,
            "base_spe": 50
        },
        {
            "name": "exeggutor",
            "level": 100,
            "types": [
                "grass",
                "psychic"
            ],
            "base_hp": 

Codice per creare nuove variabili:
- Percentuale di turni in cui il pokemon del player presenta uno status (1 per giocatore)
- Numero totale di boosts ricevuti (1 per ogni giocatore)
- Numero di turni "null" (1 per giocatore)
- Media accuracy (1 per giocatore)
- Numero di volte in cui un giocatore attacca (1 per giocatore)
- Somma della priority (1 per giocatore)

In [5]:

types = [
    "bug", "dark", "dragon", "electric", "fairy", "fighting", "fire", "flying",
    "ghost", "grass", "ground", "ice", "normal", "poison", "psychic", "rock",
    "steel", "stellar", "water"
]

# Matrice delle efficacie (valori numerici)
# x2 = 2.0, x½ = 0.5, x0 = 0.0, x1 = 1.0
# Ogni riga corrisponde al tipo attaccante, ogni colonna al tipo difensore
type_chart = np.array([
#  Bu  Da  Dr  El  Fa  Fi  Fi  Fl  Gh  Gr  Gr  Ic  No  Po  Ps  Ro  St  St  Wa
  [1,  2,  1,  1,  1,  0.5,0.5,0.5,1,  2,  1,  1,  1,  0.5,2,  1,  0.5,1,  1],  # Bug
  [1,  0.5,1,  1,  2,  1,  1,  1,  2,  1,  1,  1,  1,  1,  0,  1,  1,  1,  1],  # Dark
  [1,  1,  2,  1,  0,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  0.5,1,  1],  # Dragon
  [1,  1,  0.5,0.5,1,  1,  1,  2,  1,  0.5,0,  1,  1,  1,  1,  1,  1,  1,  2],  # Electric
  [1,  2,  2,  1,  1,  0.5,0.5,1,  1,  1,  1,  1,  1,  0.5,1,  1,  0.5,1,  1],  # Fairy
  [0.5,2,  1,  1,  0.5,1,  1,  0,  1,  1,  1,  2,  1,  0.5,2,  2,  1,  1,  1],  # Fighting
  [0.5,1,  0.5,1,  1,  1,  0.5,1,  1,  2,  1,  2,  1,  1,  1,  0.5,0.5,1,  0.5],# Fire
  [2,  1,  1,  0.5,1,  2,  1,  1,  1,  2,  1,  1,  1,  1,  1,  0.5,0.5,1,  1],  # Flying
  [1,  0.5,1,  1,  1,  1,  1,  1,  2,  1,  1,  1,  0,  1,  2,  1,  1,  1,  1],  # Ghost
  [0.5,1,  1,  1,  1,  1,  0.5,0.5,1,  0.5,2,  1,  1,  0.5,1,  2,  0.5,1,  0.5],# Grass
  [0.5,1,  1,  2,  1,  1,  2,  0,  1,  0.5,1,  2,  1,  2,  1,  2,  1,  1,  1],  # Ground
  [1,  1,  2,  1,  1,  1,  0.5,2,  1,  2,  1,  0.5,1,  1,  1,  1,  0.5,1,  0.5],# Ice
  [1,  1,  1,  1,  1,  1,  1,  1,  0,  1,  1,  1,  1,  1,  1,  0.5,0.5,1,  1],  # Normal
  [1,  1,  1,  1,  1,  1,  1,  1,  1,  2,  0.5, 1,  1,  0.5,1,  0.5,0,  1,  1], # Poison
  [1,  2,  1,  1,  1,  0.5,1,  1,  1,  1,  1,  1,  1,  1,  0.5, 1,  0.5,1,  1], # Psychic
  [2,  1,  1,  1,  1,  0.5,2,  2,  1,  1,  0.5, 2,  1,  1,  1,  1,  0.5,1,  1], # Rock
  [1,  1,  0.5,0.5,1,  1,  2,  1,  1,  1,  2,  2,  1,  1,  1,  2,  0.5,1,  0.5],# Steel
  [1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1],  # Stellar
  [1,  1,  1,  1,  1,  1,  2,  1,  1,  2,  1,  0.5,1,  1,  1,  2,  1,  1,  0.5] # Water
])

# Converti in DataFrame per etichette leggibili
type_chart_df = pd.DataFrame(type_chart, index=types, columns=types)


In [ ]:
count_level = 0
pokemon_not_level_100 = []
for dataset in [train_data, test_data]:
    for battle in dataset:
        squad = battle.get("p1_team_details")
        for pokemon in squad:
            level = pokemon.get("level")
            if level != 100:
                print(pokemon.get("name"), pokemon.get("level"))
                count_level += 1
                pokemon_not_level_100.append(pokemon)
#print(count_level)
#print(pokemon_not_level_100)

In [ ]:
used_pokemon = set()
count = 0
for battle in train_data:
    battle_timeline = battle.get('battle_timeline', [])
    for i in battle_timeline:
        nome_p1 = i.get("p1_pokemon_state").get("name")
        nome_p2 = i.get("p2_pokemon_state").get("name")
        if nome_p1 not in used_pokemon:
            used_pokemon.add(nome_p1)
            count += 1
        if nome_p2 not in used_pokemon:
            used_pokemon.add(nome_p2)
            count += 1

for battle in test_data:
    battle_timeline = battle.get('battle_timeline', [])
    for i in battle_timeline:
        nome_p1 = i.get("p1_pokemon_state").get("name")
        nome_p2 = i.get("p2_pokemon_state").get("name")
        if nome_p1 not in used_pokemon:
            used_pokemon.add(nome_p1)
            count += 1
        if nome_p2 not in used_pokemon:
            used_pokemon.add(nome_p2)
            count += 1

print(count)

In [6]:
pokedex = {}

for battle in train_data:
    squad = battle.get("p1_team_details")
    for pokemon in squad:
        nome = pokemon.get("name")
        if nome not in pokedex:
            pokedex[nome] = pokemon
    pokemon_p2 = battle.get("p2_lead_details").get("name")
    if pokemon_p2 not in pokedex:
        pokedex[pokemon_p2] = battle.get("p2_lead_details")

for battle in test_data:
    battle_timeline = battle.get('battle_timeline', [])
    squad = battle.get("p1_team_details")
    for pokemon in squad:
        nome = pokemon.get("name")
        if nome not in pokedex:
            pokedex[nome] = pokemon
    pokemon_p2 = battle.get("p2_lead_details").get("name")
    if pokemon_p2 not in pokedex:
        pokedex[pokemon_p2] = battle.get("p2_lead_details")

print(len(pokedex))
print(json.dumps(pokedex, indent=4))

20
{
    "starmie": {
        "name": "starmie",
        "level": 100,
        "types": [
            "psychic",
            "water"
        ],
        "base_hp": 60,
        "base_atk": 75,
        "base_def": 85,
        "base_spa": 100,
        "base_spd": 100,
        "base_spe": 115
    },
    "exeggutor": {
        "name": "exeggutor",
        "level": 100,
        "types": [
            "grass",
            "psychic"
        ],
        "base_hp": 95,
        "base_atk": 95,
        "base_def": 85,
        "base_spa": 125,
        "base_spd": 125,
        "base_spe": 55
    },
    "chansey": {
        "name": "chansey",
        "level": 100,
        "types": [
            "normal",
            "notype"
        ],
        "base_hp": 250,
        "base_atk": 5,
        "base_def": 5,
        "base_spa": 105,
        "base_spd": 105,
        "base_spe": 50
    },
    "snorlax": {
        "name": "snorlax",
        "level": 100,
        "types": [
            "normal",
            "n

In [ ]:
from tqdm.notebook import tqdm
import numpy as np

def create_simple_features(data: list[dict]) -> pd.DataFrame:
    """
    A very basic feature extraction function.
    It only uses the aggregated base stats of the player's team and opponent's lead.
    """
    feature_list = []
    for battle in tqdm(data, desc="Extracting features"):
        features = {}
        
        # --- Player 1 Team Features ---
        p1_team = battle.get('p1_team_details', [])
        if p1_team:
            features['p1_mean_hp'] = np.mean([p.get('base_hp', 0) for p in p1_team])
            features['p1_mean_spe'] = np.mean([p.get('base_spe', 0) for p in p1_team])
            features['p1_mean_atk'] = np.mean([p.get('base_atk', 0) for p in p1_team])
            features['p1_mean_def'] = np.mean([p.get('base_def', 0) for p in p1_team])
            features['p1_mean_spa'] = np.mean([p.get('base_spa', 0) for p in p1_team])
            features['p1_mean_spd'] = np.mean([p.get('base_spd', 0) for p in p1_team])

        # --- Player 2 Lead Features ---
        #p2_lead = battle.get('p2_lead_details')
        #if p2_lead:
            # Player 2's lead Pokémon's stats
            #features['p2_lead_hp'] = p2_lead.get('base_hp', 0)
            #features['p2_lead_spe'] = p2_lead.get('base_spe', 0)
            #features['p2_lead_atk'] = p2_lead.get('base_atk', 0)
            #features['p2_lead_def'] = p2_lead.get('base_def', 0)
            #features['p2_lead_spa'] = p2_lead.get('base_spa', 0)
            #features['p2_lead_spd'] = p2_lead.get('base_spd', 0)

        battle_timeline = battle.get('battle_timeline', [])
        p2_team = {}
        for i in battle_timeline:
            name_p2 = i.get("p2_pokemon_state", {}).get("name")
            if name_p2:
                p2_team[name_p2] = pokedex.get(name_p2, {})
        features['p2_mean_hp'] = np.mean([p2_team[p].get('base_hp', 0) for p in p2_team])
        features['p2_mean_spe'] = np.mean([p2_team[p].get('base_spe', 0) for p in p2_team])
        features['p2_mean_atk'] = np.mean([p2_team[p].get('base_atk', 0) for p in p2_team])
        features['p2_mean_def'] = np.mean([p2_team[p].get('base_def', 0) for p in p2_team])
        features['p2_mean_spa'] = np.mean([p2_team[p].get('base_spa', 0) for p in p2_team])
        features['p2_mean_spd'] = np.mean([p2_team[p].get('base_spd', 0) for p in p2_team])

        # --- Players Status pokemon ---
        
        #if battle_timeline:
        status_p1=0
        status_p2=0
        for i in battle_timeline:
            if i.get('p1_pokemon_state').get("status")!= 'nostatus':
                status_p1+=1
            if i.get('p2_pokemon_state').get("status")!= 'nostatus':
                status_p2+=1
        status_p1= status_p1/len(battle_timeline)
        features['p1_status'] =  round(status_p1,3)
        status_p2= status_p2/len(battle_timeline)
        features['p2_status'] =  round(status_p2,3)

                # --- Players Boosts ---
                
        boosts_p1 = 0
        boosts_p2 = 0
        
        for i in battle_timeline:
            boosts1 = i.get('p1_pokemon_state').get("boosts")
            boosts2 = i.get('p2_pokemon_state').get("boosts")
        
            for j in boosts1:
                boosts_p1 += boosts1.get(j)
            for k in boosts2:
                boosts_p2 += boosts2.get(k)
        
        
        #features["boosts_p1"] = boosts_p1
        #features["boosts_p2"] = boosts_p2
        features["diff_boost"] = boosts_p1 - boosts_p2
        
                # --- Players' move "null" ---
        null_1=0
        null_2=0
        for i in battle_timeline:
            if not i.get('p1_move_details'):
                null_1 += 1
            if not i.get('p2_move_details'):
                null_2 += 1
        
        features['null_p1'] = null_1
        features['null_p2'] = null_2

                # --- Players' accuracy ---
        acc_1 = 0
        n_acc1 = 0
        acc_2 = 0
        n_acc2 = 0
        for i in battle_timeline:
            if i.get('p1_move_details'):
                acc_1 += float(i.get('p1_move_details').get('accuracy'))
                n_acc1 += 1
            if i.get('p2_move_details'):
                acc_2 += float(i.get('p2_move_details').get('accuracy'))
                n_acc2 += 1
        '''if n_acc1 != 0:
            features['avg_acc_p1'] = acc_1/n_acc1
        else:
            features['avg_acc_p1'] = 0
        if n_acc2 != 0:
            features['avg_acc_p2'] = acc_2/n_acc2
        else:
            features['avg_acc_p2'] = 0
                
        features["diff_avg_acc"] = features['avg_acc_p1'] - features['avg_acc_p2']'''

        if n_acc1 != 0:
            avg_acc1 = acc_1/n_acc1
        else:
            avg_acc1 = 0
        if n_acc2 != 0:
            avg_acc2 = acc_2/n_acc2
        else:
            avg_acc2 = 0

        features["diff_avg_acc"] = avg_acc1 - avg_acc2

        
        # numero di volte in cui attacca

        diz_p1 = {}
        count_p2 = 0
        for i in battle_timeline:
            nome = i.get('p1_pokemon_state').get('name')
            hp = i.get('p1_pokemon_state').get('hp_pct')
            if nome not in diz_p1:
                if int(hp) != 1:
                    diz_p1[nome] = hp
                    count_p2 += 1
                else:
                    diz_p1[nome] = hp
            else:
                if diz_p1[nome] > hp:
                    diz_p1[nome] = hp
                    count_p2 += 1
                else:
                    diz_p1[nome] = hp
        diz_p2 = {}
        count_p1 = 0
        for i in battle_timeline:
            nome = i.get('p2_pokemon_state').get('name')
            hp = i.get('p2_pokemon_state').get('hp_pct')
            if nome not in diz_p2:
                if int(hp) != 1:
                    diz_p2[nome] = hp
                    count_p1 += 1
                else:
                    diz_p2[nome] = hp
            else:
                if diz_p2[nome] > hp:
                    diz_p2[nome] = hp
                    count_p1 += 1
                else:
                    diz_p2[nome] = hp

        features['n_atk_p1'] = count_p1
        features['n_atk_p2'] = count_p2

        
        # Diff damage inflicted

        diz_1 = {}
        def_p1 = 0
        diz_2 = {}
        atk_p1 = 0
        for i in battle_timeline:
            # somma difesa p1
            diff_p1 = 0
            nome_1 = i.get("p1_pokemon_state").get("name")
            hp_1 = i.get("p1_pokemon_state").get("hp_pct")
            if nome_1 not in diz_1:
                if int(hp_1) != 1:
                    diff_p1 = 1 - hp_1
                    diz_1[nome_1] = hp_1
                else:
                    diz_1[nome_1] = hp_1
            else:
                diff_p1 = diz_1[nome_1] - hp_1
                diz_1[nome_1] = hp_1
            def_p1 += diff_p1 
            
        for i in battle_timeline:
            # somma attacco p1 (o difesa p2)
            diff_p2 = 0
            nome_2 = i.get("p2_pokemon_state").get("name")
            hp_2 = i.get("p2_pokemon_state").get("hp_pct")   
            if nome_2 not in diz_2:
                if int(hp_2) != 1:
                    diff_p2 = 1 - hp_2
                    diz_2[nome_2] = hp_2
                else:
                    diz_2[nome_2] = hp_2
            else:
                diff_p2 = diz_2[nome_2] - hp_2
                diz_2[nome_2] = hp_2
            atk_p1 += diff_p2
        
        features["diff_damage"] = atk_p1 - def_p1
        
        # Count priority
        
        priority_1 = 0
        for i in battle_timeline:
            if i.get('p1_move_details'):
                priority = i.get('p1_move_details').get('priority')
                priority_1 += priority

        priority_2 = 0
        for i in battle_timeline:
            if i.get('p2_move_details'):
                priority = i.get('p2_move_details').get('priority')
                priority_2 += priority
        features["priority_1"] = priority_1
        features["priority_2"] = priority_2
        #features["diff_priority"] = priority_1 - priority_2


             # --- Players' offensive effectivness ---

        types_p1 = {}
        p1_team = battle.get('p1_team_details')
        for p in p1_team:
            types_p1[p.get('name')] = p.get('types')
        
        
        count_1 = 0
        for i in battle_timeline:
            molt = 1
            if i.get('p2_move_details') and i.get('p2_move_details').get('category') != "STATUS":
                move = i.get('p2_move_details').get('type').lower()
                name_p1 = i.get('p1_pokemon_state').get('name')
                for j in types_p1[name_p1]:
                    if j == "notype":
                        pass
                    else:
                        if move != "notype":
                            molt *= type_chart_df.loc[move,j]
                if molt > 1:
                    count_1 += 1
        features["oe_p2"] = count_1

        types_p2 = {}
        count_2 = 0
        for i in battle_timeline:
            name_p2 = i.get("p2_pokemon_state").get("name")
            if name_p2 not in types_p2:
                types_p2[name_p2] = pokedex[name_p2].get("types")

        for i in battle_timeline:
            molt = 1
            if i.get('p1_move_details') and i.get('p1_move_details').get('category') != "STATUS":
                move = i.get('p1_move_details').get('type').lower()
                name_p2 = i.get('p2_pokemon_state').get('name')
                for j in types_p2[name_p2]:
                    if j == "notype":
                        pass
                    else:
                        if move != "notype":
                            molt *= type_chart_df.loc[move,j]
                if molt > 1:
                    count_2 += 1
        features["oe_p1"] = count_2

        #features["diff_oe"] = count_1 - count_2
                

                # --- Players' KO ---

        count_p1 = 0
        count_p2 = 0
        for i in battle_timeline:
            if i.get('p1_pokemon_state').get('status') == 'fnt':
                count_p1 += 1
            if i.get('p2_pokemon_state').get('status') == 'fnt':
                count_p2 += 1
        features['ko_p1'] = count_p1
        features['ko_p2'] = count_p2
        #features["diff_ko"] = count_p1 - count_p2


                # --- Number of special attacks ---
        
        count_p1 = 0
        sp_atk_1 = 0
        count_p2 = 0
        sp_def_1 = 0
        for i in battle_timeline:
            if i.get("p1_move_details"):
                nome_p1 = i.get("p1_pokemon_state").get("name")
                if i.get('p1_move_details').get('category') == 'SPECIAL':
                    count_p1 += 1
                    sp_atk_1 += next(p.get('base_spa', 0) for p in p1_team if p.get("name") == nome_p1)
            if i.get("p2_move_details"):
                if i.get('p2_move_details').get('category') == 'SPECIAL':
                    count_p2 += 1
        features['n_sp_atk_1'] = count_p1
        features['n_sp_atk_2'] = count_p2
        #features["diff_sp_atk"] = count_p1 - count_p2
        '''if count_p1 != 0:
            features["avg_sp_atk_1"] = sp_atk_1 / count_p1
        else:
            features["avg_sp_atk_1"] = 0
        features["sp_atk_1"] = sp_atk_1'''
        
        


        #features["null_p1 x ko_p1"] = features["null_p1"]*features["ko_p1"]
        #features["n_atk_p2 x ko_p1"] = features["n_atk_p2"]*features["ko_p1"]
        #features["p2_status x null_p2"] = features["p2_status"]*features["null_p2"]
        features["p2_status x null_p2"] = features["p2_status"]*null_2
        #features["null_p1 x n_atk_p2"] = features["null_p1"]*features["n_atk_p2"]

        
        #features["p2_lead_spe x n_atk_p2 x ko_p1"] = features["p2_lead_spe"]*features["n_atk_p2"]*features["ko_p1"]
        #features["p2_lead_spe x p2_status x null_p2"] = features["p2_lead_spe"]*features["p2_status"]*features["null_p2"]
        #features["p2_lead_spe x null_p1 x n_atk_p2"] = features["p2_lead_spe"]*features["null_p1"]*features["n_atk_p2"]
        #features["p2_lead_spe x null_p1 x ko_p1"] = features["p2_lead_spe"]*features["null_p1"]*features["ko_p1"]
        #features["p2_lead_atk x null_p2"] = features["p2_lead_atk"]*features["null_p2"]
        #features["p2_lead_atk x null_p1 x ko_p1"] = features["p2_lead_atk"]*features["null_p1"]*features["ko_p1"]
        #features["p2_lead_atk x n_atk_p2 x ko_p1"] = features["p2_lead_atk"]*features["n_atk_p2"]*features["ko_p1"]
        #features["p2_lead_def x null_p2"] = features["p2_lead_def"]*features["null_p2"]
        #features["p2_lead_def x n_atk_p2 x ko_p1"] = features["p2_lead_def"]*features["n_atk_p2"]*features["ko_p1"]
        #features["p2_lead_def x null_p2 x p2_status"] = features["p2_lead_def"]*features["null_p2"]*features["p2_status"]
        #features["p1_status x null_p1"] = features["p1_status"]*features["null_p1"]
        features["p1_status x null_p1"] = features["p1_status"]*null_1
        #features["p1_status x null_p1 x ko_p1"] = features["p1_status"]*features["null_p1"]*features["ko_p1"]
        #features["p1_status x n_atk_p2 x ko_p1"] = features["p1_status"]*features["n_atk_p2"]*features["ko_p1"]
        #features["p1_status x null_p1 x n_atk_p2"] = features["p1_status"]*features["null_p1"]*features["n_atk_p2"]
        #features["n_atk_p2 x null_p1 x ko_p1"] = features["n_atk_p2"]*features["null_p1"]*features["ko_p1"]
        #features["oe_p1 x null_p2 x p2_status"] = features["oe_p1"]*features["null_p2"]*features["p2_status"]


            
        # We also need the ID and the target variable (if it exists)
        features['battle_id'] = battle.get('battle_id')
        if 'player_won' in battle:
            features['player_won'] = int(battle['player_won'])
            
        feature_list.append(features)
        
    return pd.DataFrame(feature_list).fillna(0)

# Create feature DataFrames for both training and test sets
#print("Processing training data...")
train_df = create_simple_features(train_data)

#print("\nProcessing test data...")

test_df = create_simple_features(test_data)

#print("\nTraining features preview:")
display(train_df.head(5))
display(train_df.tail(5))

train_df.describe()

In [ ]:
from sklearn.linear_model import LogisticRegression

# Define our features (X) and target (y)
features = [col for col in train_df.columns if col not in ['battle_id', 'player_won']]
X_train = train_df[features]
y_train = train_df['player_won']

X_test = test_df[features]

# Initialize and train the model
#print("Training a simple Logistic Regression model...")

#fit1 = model.fit(X_train, y_train)
#print("Model training complete.")

In [ ]:
'''import matplotlib.pyplot as plt
from itertools import combinations

for f1, f2 in combinations(features[13:], 2):
    plt.figure()
    plt.scatter(X_train[f1]*X_train[f2], y_train + np.random.uniform(-0.5, 0.5, size=len(y_train)), c = y_train, cmap='bwr', alpha=0.7)
    plt.xlabel(f"{f1} x {f2}")
    plt.ylabel("player_won")
    plt.title(f'Scatterplot: {f1} x {f2}')
    plt.grid(True)
    plt.show()'''

**Outliers analysis**

In [ ]:
# [Q1 - 1.5(Q3-Q1), Q3 + 1.5(Q3-Q1)

'''for i in X_train:
    q1 = X_train[i].quantile(0.25)
    q3 = X_train[i].quantile(0.75)
    
    lb = q1 - 1.5*(q3-q1)
    up = q3 + 1.5*(q3-q1)
    count = 0
    for j in X_train[i]:
        if j < lb or j > up:
            count += 1
    print(f"range {i} = [{lb},{up}], count = {count}")'''

**Scaling**

In [ ]:
from sklearn import metrics
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.fit_transform(X_test)



X_train_scaled_df = pd.DataFrame(X_train_scaled, columns= features )
X_train_scaled_df.head()
X_train_scaled_df.describe()

In [ ]:
lista = []
for i in features:
    corr = X_train_scaled_df[i].corr(y_train)
    if abs(corr) < 0.05:
        lista.append(i)
print(lista)
    

**Grid Search**

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, roc_auc_score

model = LogisticRegression(random_state=42, max_iter=1000)

# Define the parameter grid to search
param_grid = {
    'C': [0.1, 0.5, 1, 1.5, 2],
    'penalty': ['l1', 'l2'],
    'solver': ['liblinear', 'lbfgs']
}

# Create the GridSearchCV object
grid_logreg = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    scoring=['roc_auc', 'accuracy'],
    n_jobs=4,        # use 4 cores in parallel
    cv=5,            # 5-fold cross-validation, more on this later
    refit='roc_auc',      # retrain the best model on the full training set
    return_train_score=True
)

# Fit the GridSearchCV object on the training data
grid_logreg.fit(X_train_scaled, y_train)

# Read the cv_results_ property into a DataFrame
cv_results_df = pd.DataFrame(grid_logreg.cv_results_)

# Check what type of object the best_estimator_ property returns
print(type(grid_logreg.best_estimator_))

# Use the best estimator to make predictions on the test set
predictions = grid_logreg.best_estimator_.predict(X_test_scaled)

# Display a few predicted class labels (0 = benign, 1 = malignant)
print("Predicted labels:", predictions[:10])

In [ ]:

best_score = grid_logreg.best_score_
print("Best ROC_AUC score:", best_score) # output: the biggest AUC w.r.t. mean_test_score 


cv_df = pd.DataFrame(grid_logreg.cv_results_)
best_idx = grid_logreg.best_index_
mean_acc = cv_df.loc[best_idx, 'mean_test_accuracy']
print("\naccuracy",mean_acc) 

mean_auc = cv_df.loc[best_idx, 'mean_test_roc_auc']
print("\nauc",mean_auc) 

best_params = grid_logreg.best_params_
print("\nBest hyperparameters:\n",best_params) # ouput: hyperparameters used in the best model

# Estrai il modello migliore
best_log_reg = grid_logreg.best_estimator_

# Coefficienti del modello (per una regressione logistica)

coef = best_log_reg.coef_.flatten()

feature_names = X_train.columns

coef_df = pd.DataFrame({
    'Feature': feature_names,
    'Coefficient': abs(coef)
}).sort_values(by='Coefficient', ascending=False)

print(coef_df)

# auc = 0.9008
# acc = 0.8296
# {'C': 0.5, 'penalty': 'l1', 'solver': 'liblinear'}

**KNN**

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

ks = range(1, 30)
# Compute 5-fold cross-validation accuracy for each k
scores = [
    cross_val_score(KNeighborsClassifier(n_neighbors=k), X_train_scaled, y_train, cv=5).mean()
    for k in ks
]
plt.plot(ks, scores, marker='o')
plt.xlabel('k (number of neighbors)')
plt.ylabel('Cross-Validation Accuracy')
plt.title('Choosing the Optimal k in KNN')
plt.grid(True)
plt.show()

#elbow at k = 14 

In [ ]:
acc_knn = cross_val_score(KNeighborsClassifier(n_neighbors = 14), X_train_scaled, y_train, cv = 5).mean()
print(acc_knn)

**Decision tree**

In [ ]:
from sklearn.tree import DecisionTreeClassifier, plot_tree

'''full_tree = DecisionTreeClassifier(random_state=42)
full_tree.fit(X_train_scaled, y_train)

# Compute effective alphas and impurities for pruning
path = full_tree.cost_complexity_pruning_path(X_train_scaled, y_train)
ccp_alphas, impurities = path.ccp_alphas, path.impurities

# Evaluate performance for each alpha using cross-validation
acc_scores = []
for ccp_alpha in ccp_alphas:
    pruned_tree = DecisionTreeClassifier(random_state=42, ccp_alpha=ccp_alpha)
    score = cross_val_score(pruned_tree, X_train_scaled, y_train, cv=5).mean()
    acc_scores.append(score)

# Plot cross-validation accuracy vs alpha
plt.figure(figsize=(6, 4))
plt.plot(ccp_alphas, acc_scores, marker='o', drawstyle="steps-post")
plt.xlabel("ccp_alpha")
plt.ylabel("Cross-Validation Accuracy")
plt.title("Cost Complexity Pruning")
plt.grid(True)
plt.show()'''

In [ ]:
'''best_alpha = ccp_alphas[np.argmax(acc_scores)]
print(f"Optimal alpha: {best_alpha:.5f}")
best_tree = DecisionTreeClassifier(criterion = "gini", random_state = 42, ccp_alpha = best_alpha)
tree_acc = cross_val_score(best_tree, X_train_scaled, y_train, cv = 5).mean()
print(tree_acc)'''

**XGboost, ADABoost**

In [ ]:
pip install xgboost -q

In [ ]:
from sklearn.ensemble import VotingClassifier, BaggingClassifier, RandomForestClassifier, AdaBoostClassifier

In [ ]:
from xgboost import XGBClassifier
ada_clf = AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=1),  # weak learner (stump)
    n_estimators=100,
    learning_rate=0.5,
    random_state=42
)
acc_ada = cross_val_score(ada_clf, X_train_scaled, y_train, cv =5).mean()
xgb_clf = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
acc_xgb = cross_val_score(xgb_clf, X_train_scaled, y_train, cv =5).mean()
print(acc_ada,"\n", acc_xgb )

**Random forest, Bagging**

In [ ]:
rf_clf = RandomForestClassifier(n_estimators=100, random_state=42)

acc_rf = cross_val_score(rf_clf, X_train_scaled, y_train, cv = 5).mean()
print(acc_rf)

bag_clf = BaggingClassifier(
    DecisionTreeClassifier(),
    n_estimators=100,       # number of trees
    max_samples=0.8,        # each tree sees 80% of training data
    bootstrap=True,         # sample with replacement
    random_state=42
)
acc_bag = cross_val_score(bag_clf, X_train_scaled, y_train, cv = 5).mean()
print(acc_bag)

**Ensamble (Voting)**

In [ ]:
log_clf = best_log_reg
knn_clf = KNeighborsClassifier(n_neighbors = 18)

# voting con random forest
voting_rf = VotingClassifier(
    estimators=[
        ('lr', log_clf),
        ('rf', rf_clf),
        ('knn', knn_clf)
    ],
    voting='soft'  # --> average predicted probabilities
)
voting_acc_rf = cross_val_score(voting_rf, X_train_scaled, y_train, cv = 5).mean()


# voting con ada_boost
voting_ada = VotingClassifier(
    estimators=[
        ('lr', log_clf),
        ('ada', ada_clf),
        ('knn', knn_clf)
    ],
    voting='soft'  # --> average predicted probabilities
)
voting_acc_ada = cross_val_score(voting_ada, X_train_scaled, y_train, cv = 5).mean()

# voting con xgboost
voting_xgb = VotingClassifier(
    estimators=[
        ('lr', log_clf),
        ('xgb', xgb_clf),
        ('knn', knn_clf)
    ],
    voting='soft'  # --> average predicted probabilities
)
voting_acc_xgb = cross_val_score(voting_xgb, X_train_scaled, y_train, cv = 5).mean()

# voting con xgboost e rf
voting_xgb_rf = VotingClassifier(
    estimators=[
        ('lr', log_clf),
        ('xgb', xgb_clf),
        ('knn', knn_clf),
        ('rf', rf_clf)
    ],
    voting='soft'  # --> average predicted probabilities
)
voting_acc_xgb_rf = cross_val_score(voting_xgb_rf, X_train_scaled, y_train, cv = 5).mean()

print(f"accuracy con random forest {voting_acc_rf}")
print(f"accuracy con ada_boost {voting_acc_ada}")
print(f"accuracy con xgboost {voting_acc_xgb}")
print(f"accuracy con xgboost e rf {voting_acc_xgb_rf}")

**model stacking**

In [ ]:
from sklearn.ensemble import StackingClassifier

# con random forest

models = [
    ("lr", log_clf),
    ("rf", rf_clf),
    ("knn", knn_clf)
]

meta_learner = LogisticRegression(random_state = 42, max_iter = 1000)
stacking_rf = StackingClassifier(
    estimators=models,
    final_estimator=meta_learner,
    cv=5  # cross-validation for base model predictions
)

In [ ]:
param_grid = {
    'final_estimator__C': [0.1, 1, 1.5, 2],
    'final_estimator__penalty': ['l1', 'l2'],
    'final_estimator__solver': ['liblinear', 'lbfgs']
}

grid_rf = GridSearchCV(stacking_rf, param_grid, scoring = ["roc_auc", "accuracy"], refit = "roc_auc",n_jobs = 4 , cv=5)
grid_rf.fit(X_train_scaled, y_train)
predictions = grid_rf.best_estimator_.predict(X_test_scaled)


rf_results_df = pd.DataFrame(grid_rf.cv_results_)
best_idx = grid_rf.best_index_
acc_stacking_rf = rf_results_df.loc[best_idx, 'mean_test_accuracy']

In [ ]:
# con ada_boost

models = [
    ("lr", log_clf),
    ("ada", ada_clf),
    ("knn", knn_clf)
]

meta_learner = LogisticRegression(random_state = 42, max_iter = 1000) # o best_log_reg?

stacking_ada = StackingClassifier(
    estimators=models,
    final_estimator=meta_learner,
    cv=5  # cross-validation for base model predictions
)

#acc_stacking_ada = cross_val_score(stacking_ada, X_train_scaled, y_train, cv = 5).mean()


In [ ]:
param_grid = {
    'final_estimator__C': [0.1, 1, 1.5, 2],
    'final_estimator__penalty': ['l1', 'l2'],
    'final_estimator__solver': ['liblinear', 'lbfgs']
}

grid_ada = GridSearchCV(stacking_ada, param_grid, scoring = ["roc_auc", "accuracy"], refit = "roc_auc",n_jobs = 4 , cv=5)
grid_ada.fit(X_train_scaled, y_train)
predictions = grid_ada.best_estimator_.predict(X_test_scaled)


ada_results_df = pd.DataFrame(grid_ada.cv_results_)
best_idx = grid_ada.best_index_
acc_stacking_ada = ada_results_df.loc[best_idx, 'mean_test_accuracy']

In [ ]:
# con xgboost

models = [
    ("lr", log_clf),
    ("xgb", xgb_clf),
    ("knn", knn_clf)
]

meta_learner = LogisticRegression(random_state = 42, max_iter = 1000) # o best_log_reg?

stacking_xgb = StackingClassifier(
    estimators=models,
    final_estimator=meta_learner,
    cv=5  # cross-validation for base model predictions
)

#acc_stacking_xgb = cross_val_score(stacking_xgb, X_train_scaled, y_train, cv = 5).mean()


In [ ]:
param_grid = {
    'final_estimator__C': [0.1, 1, 1.5, 2],
    'final_estimator__penalty': ['l1', 'l2'],
    'final_estimator__solver': ['liblinear', 'lbfgs']
}

grid_xgb = GridSearchCV(stacking_xgb, param_grid, scoring = ["roc_auc", "accuracy"], refit = "roc_auc",n_jobs = 4 , cv=5)
grid_xgb.fit(X_train_scaled, y_train)
predictions = grid_xgb.best_estimator_.predict(X_test_scaled)


xgb_results_df = pd.DataFrame(grid_xgb.cv_results_)
best_idx = grid_xgb.best_index_
acc_stacking_xgb = xgb_results_df.loc[best_idx, 'mean_test_accuracy']

In [ ]:
# con xgboost e random forest

models = [
    ("lr", log_clf),
    ("xgb", xgb_clf),
    ("knn", knn_clf),
    ('rf', rf_clf)
]

meta_learner = LogisticRegression(random_state = 42, max_iter = 1000) # o best_log_reg?

stacking_xgb_rf = StackingClassifier(
    estimators=models,
    final_estimator=meta_learner,
    cv=5  # cross-validation for base model predictions
)

In [ ]:
param_grid = {
    'final_estimator__C': [0.1, 1, 1.5, 2],
    'final_estimator__penalty': ['l1', 'l2'],
    'final_estimator__solver': ['liblinear', 'lbfgs']
}

grid_xgb_rf = GridSearchCV(stacking_xgb_rf, param_grid, scoring = ["roc_auc", "accuracy"], refit = "roc_auc",n_jobs = 4 , cv=5)
grid_xgb_rf.fit(X_train_scaled, y_train)
predictions = grid_xgb_rf.best_estimator_.predict(X_test_scaled)


xgb_rf_results_df = pd.DataFrame(grid_xgb_rf.cv_results_)
best_idx = grid_xgb_rf.best_index_
acc_stacking_xgb_rf = xgb_rf_results_df.loc[best_idx, 'mean_test_accuracy']

In [ ]:
print(f"accuracy con rf {acc_stacking_rf}")
print(f"accuracy con ada_boost {acc_stacking_ada}")
print(f"accuracy con xgboost {acc_stacking_xgb}")
print(f"accuracy con xgboost e rf {acc_stacking_xgb_rf}")

### 4. Creating the Submission File

The competition requires a `.csv` file with two columns: `battle_id` and `player_won`. Let's use our trained model to make predictions on the test set and format them correctly.

In [ ]:
# Make predictions on the test data
print("Generating predictions on the test set...")
#test_predictions = model.predict(X_test)

# Create the submission DataFrame
submission_df = pd.DataFrame({
    'battle_id': test_df['battle_id'],
    #'player_won': test_predictions
    'player_won': predictions
})

# Save the DataFrame to a .csv file
submission_df.to_csv('submission.csv', index=False)

print("\n'submission.csv' file created successfully!")
display(submission_df.head())

### 5. Submitting Your Results

Once you have generated your `submission.csv` file, there are two primary ways to submit it to the competition.

---

#### Method A: Submitting Directly from the Notebook

This is the standard method for code competitions. It ensures that your submission is linked to the code that produced it, which is crucial for reproducibility.

1.  **Save Your Work:** Click the **"Save Version"** button in the top-right corner of the notebook editor.
2.  **Run the Notebook:** In the pop-up window, select **"Save & Run All (Commit)"** and then click the **"Save"** button. This will run your entire notebook from top to bottom and save the output, including your `submission.csv` file.
3.  **Go to the Viewer:** Once the save process is complete, navigate to the notebook viewer page. 
4.  **Submit to Competition:** In the viewer, find the **"Submit to Competition"** section. This is usually located in the header of the output section or in the vertical "..." menu on the right side of the page. Clicking the **Submit** button this will submit your generated `submission.csv` file.

After submitting, you will see your score in the **"Submit to Competition"** section or in the [Public Leaderboard](https://www.kaggle.com/competitions/fds-pokemon-battles-prediction-2025/leaderboard?).

---

#### Method B: Manual Upload

You can also generate your predictions and submission file using any environment you prefer (this notebook, Google Colab, or your local machine).

1.  **Generate the `submission.csv` file** using your model.
2.  **Download the file** to your computer.
3.  **Navigate to the [Leaderboard Page](https://www.kaggle.com/competitions/fds-pokemon-battles-prediction-2025/leaderboard?)** and click on the **"Submit Predictions"** button.
4.  **Upload Your File:** Drag and drop or select your `submission.csv` file to upload it.

This method is quick, but keep in mind that for the final evaluation, you might be required to provide the code that generated your submission.

Good luck!